Motivation:
          Pass the cell_state of the last cell in the previous episode to the next episode

Aspiration source: https://medium.com/@erikhallstrm/using-the-tensorflow-lstm-api-3-7-5f2b97ca6b73

The way in the previous 1_Stateless_LSTM does not work. You need to feed in throught a placehoder

1. define the placehoder
cell_state_t = tf.placeholder(tf.float32, [batch_size, state_size])
hidden_state_t = tf.placeholder(tf.float32, [batch_size, state_size])
init_state_t = tf.nn.rnn_cell.LSTMStateTuple(cell_state_t, hidden_state_t)

2. use the init_state (placehoder) as an arguemnt of tf.nn.rnn
cell = tf.nn.rnn_cell.BasicLSTMCell(state_size, state_is_tuple=True)
states_series, current_state = tf.nn.rnn(cell, inputs_series, init_state_t)

3. The initial state vaules which will feed into the placehoder
_current_cell_state = np.zeros((batch_size, state_size))
_current_hidden_state = np.zeros((batch_size, state_size))

4. Run an epoch of training and store the cell_state of the last cell
_total_loss, _train_step, _current_state, _predictions_series = sess.run(
    [total_loss, train_step, current_state, predictions_series],
    feed_dict={
        batchX_placeholder: batchX,
        batchY_placeholder: batchY,
        cell_state_t: _current_cell_state,
        hidden_state_t: _current_hidden_state

    })

5. update the initial state value. So the next time when you run step 4, the initial state of the first cell
   in the episode will not be zero
_current_cell_state, _current_hidden_state = _current_state

In [ ]:
Trap:
    The two placehoders don't have to be in the class body

In [ ]:
To do
    If you also want to feed the hidden state to the next episode, you need to cancatenate the hidden_state
    with the input manually. 
    
    The hidden_state is merged with input automatically between different cells within an episode

In [1]:
import tensorflow as tf
import numpy as np
state_size = 4
input_len =5
batch_size=3
num_step = 20

np.random.seed(0)
input = np.random.rand(batch_size,input_len)

In [2]:
class Agent:
    def __init__(self):
        self.rnn_cell = None
        self.initialized = False
        self.batch_size = None
        self.output = None
        self.rnn_state_t=None
    def first(self,input_t):
        self.batch_size = tf.unpack(tf.shape(input_t))[0]
        with tf.variable_scope("rnn"):
            self.rnn_cell = tf.nn.rnn_cell.BasicLSTMCell(state_size,state_is_tuple=True)
            self.rnn_state_t = self.rnn_cell.zero_state(self.batch_size,tf.float32)
        output_= self.predict(input_t)
        self.initialized = True
        return output_
    
    def predict(self,input_t):

        if self.initialized:
            tf.get_variable_scope().reuse_variables()
        with tf.variable_scope("rnn/RNN"):

            output_, current_state_t = self.rnn_cell(input_t, init_state_t)

        return output_ ,current_state_t

In [3]:
# input placehoders
input_t = tf.placeholder(tf.float32,(None,input_len))


cell_state_t = tf.placeholder(tf.float32, [batch_size, state_size])
hidden_state_t = tf.placeholder(tf.float32, [batch_size, state_size])
init_state_t = tf.nn.rnn_cell.LSTMStateTuple(cell_state_t, hidden_state_t)

# define operations
agent = Agent()
output1_t = agent.first(input_t)
output2_t = agent.predict(input_t)

# initialization operation
init_op = tf.global_variables_initializer()

saver = tf.train.Saver()

In [4]:
sess = tf.Session()
sess.run(init_op)

_current_cell_state = np.zeros((batch_size, state_size))
_current_hidden_state = np.zeros((batch_size, state_size))

output1_v1,output2_v1 = sess.run([output1_t, output2_t],{input_t: input, cell_state_t: _current_cell_state,
        hidden_state_t: _current_hidden_state})

_current_cell_state, _current_hidden_state = output2_v1[1]

save_path = saver.save(sess, './rnn_checkpoints/model.ckpt')

In [5]:
output1_v2,output2_v2 = sess.run([output1_t, output2_t],{input_t: input, cell_state_t: _current_cell_state,
        hidden_state_t: _current_hidden_state})

_current_cell_state, _current_hidden_state = output2_v1[1]

In [6]:
output1_v3,output2_v3 = sess.run([output1_t, output2_t],{input_t: input, cell_state_t: _current_cell_state,
        hidden_state_t: _current_hidden_state})


In [7]:
# This shows the initial state changes
(output2_v1[0]==output2_v2[0]).all()

False

In [8]:
#This shows if you use the initial cell state, the output is the same
(output2_v2[0] == output2_v3[0]).all()

True

In [9]:
np.save("./states_saved/cell_state.npy",_current_cell_state)
np.save("./states_saved/hidden_state.npy",_current_hidden_state)

np.save("./states_saved/output2_v.npy",output2_v3[0])